# Этап 1. Проведите исследовательский анализ (EDA)

📊 БАЗОВАЯ СТАТИСТИКА:
- Количество образцов: 3262
- Минимальная калорийность: 0.00
- Средняя калорийность: 255.01
- Медианная калорийность: 209.11
- Максимальная калорийность: 3943.33
- Стандартное отклонение: 219.64
- Коэффициент вариации: 86.13%

<img src="static/Калории.jpg" width="1200" height="400">

<img src="static/Аугментация.jpg" width="1800" height="300">

Выводы:
- Есть сильные отклонения по калориями, поэтому необходимо отсечь экстремальные выбросы и нормализовать. Они будут сильно шуметь и ухудшать MAE.
- Данные по количеству ингридиентов в блюде, а также калорийности по каждому ингридиенту отсутствуют. Поэтому толку от них немного...
- Массу блюд тоже нормализуем передаем как тензор, дополнительный признак.
- Ставку больше делаем на изображения. Аугментация слабая, поскольку цвета важны для определения блюд, продуктов.
- В ходе обучения выводиться будут метрики MAE для текстовой и графической моделей отдельно.

# Этап 2. Реализуйте пайплайн обучения

In [1]:
# src/config.py
class Config:
    # Модели
    TEXT_MODEL_NAME = "bert-base-uncased"
    IMAGE_MODEL_NAME = "tf_efficientnet_b4"

    # Гиперпараметры
    BATCH_SIZE = 16
    EPOCHS = 30
    HIDDEN_DIM = 512
    DROPOUT = 0.2
    TEXT_LR = 1e-5  # LR для текстовой модели
    IMAGE_LR = 1e-4  # LR для изображений
    HEAD_LR = 1e-3  # LR для классификатора
    SEED = 42

    # Путь сохранения лучшей модели
    SAVE_PATH = "best_model.pth"

# src/dataset.py
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image
import albumentations as A
import timm

class MultimodalDataset(Dataset):
    def __init__(self, dishes_df, ingredients_df, config, ds_type="train", target_mean=None, target_std=None, mass_mean=None, mass_std=None, use_log_target=False):
        self.dishes = dishes_df.reset_index(drop=True)
        self.ingredients = ingredients_df
        self.config = config
        self.transforms = get_transforms(config, ds_type)
        self._fix_names()
        # Нормализация таргета: используем переданные статистики (возможно в лог-шкале)
        self.mean = target_mean
        self.std = target_std
        # Нормализация массы
        self.mass_mean = mass_mean
        self.mass_std = mass_std
        self.use_log_target = use_log_target

    def _fix_names(self):
        id_to_name = dict(zip(self.ingredients["id"], self.ingredients["ingr"]))
        def map_ingrs(s):
            if pd.isna(s):
                return []
            ids = [int(i.replace("ingr_", "")) for i in s.split(";")]
            return [id_to_name.get(i, i) for i in ids]
        self.dishes["ingredients"] = self.dishes["ingredients"].apply(map_ingrs)

    def __len__(self):
        return len(self.dishes)

    def __getitem__(self, idx):
        row = self.dishes.iloc[idx]
        ingredients = ", ".join([i for i in row['ingredients']])
        text = f"List of ingredients: {ingredients}\nTotal mass: {row['total_mass']} grams."
        image_path = f"data/images/{row['dish_id']}/rgb.png"
        img = Image.open(image_path)
        img_np = np.array(img)
        aug = self.transforms(image=img_np)
        img_tensor = aug["image"]
        raw_label = row["total_calories"]
        if self.use_log_target:
            lab = np.log1p(max(0.0, raw_label))
        else:
            lab = float(raw_label)
        label_norm = (lab - self.mean) / (self.std + 1e-9)
        mass = float(row["total_mass"])
        mass_norm = (mass - (self.mass_mean if self.mass_mean is not None else 0.0)) / ((self.mass_std if self.mass_std is not None else 1.0) + 1e-9)
        return {
            "text": text,
            "image": img_tensor,
            "label": torch.tensor(label_norm, dtype=torch.float32),
            "mass": torch.tensor(mass_norm, dtype=torch.float32)
        }

def get_transforms(config, ds_type="train"):
    cfg = timm.get_pretrained_cfg(config.IMAGE_MODEL_NAME)
    max_side = max(cfg.input_size[1], cfg.input_size[2])
    if ds_type == "train":
        return A.Compose([
            A.SmallestMaxSize(max_size=max_side, p=1.0),
            A.CenterCrop(height=cfg.input_size[1], width=cfg.input_size[2]),
            A.HorizontalFlip(p=0.1),
            A.Affine(translate_percent=0.05, scale=(0.9, 1.1), rotate=(-15, 15), p=0.6),
            A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05, p=0.6),
            A.RandomBrightnessContrast(p=0.1),
            A.HueSaturationValue(p=0.1),
            A.Normalize(mean=cfg.mean, std=cfg.std),
            A.pytorch.transforms.ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.SmallestMaxSize(max_size=max_side, p=1.0),
            A.CenterCrop(height=cfg.input_size[1], width=cfg.input_size[2]),
            A.Normalize(mean=cfg.mean, std=cfg.std),
            A.pytorch.transforms.ToTensorV2(),
        ])

def collate_fn(batch, tokenizer, max_length=128):
    texts = [item["text"] for item in batch]
    images = torch.stack([item["image"] for item in batch])
    labels = torch.stack([item["label"] for item in batch])
    masses = torch.stack([item["mass"] for item in batch])
    tok = tokenizer(
        texts,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    return {
        "image": images,
        "label": labels,
        "mass": masses,
        "input_ids": tok["input_ids"],
        "attention_mask": tok["attention_mask"]
    }

# src/model.py
import torch
import torch.nn as nn
from transformers import AutoModel
import timm

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * mask, dim=1) / torch.clamp(mask.sum(dim=1), min=1e-9)

def freeze_all(model):
    for param in model.parameters():
        param.requires_grad = False

def unfreeze_last_n_layers(model, n):
    layers = None
    if hasattr(model, "encoder") and hasattr(model.encoder, "layer"):
        layers = model.encoder.layer
    else:
        base = getattr(model, "base_model", None)
        if base is not None and hasattr(base, "encoder") and hasattr(base.encoder, "layer"):
            layers = base.encoder.layer

    if layers is None:
        return

    total = len(layers)
    for layer in layers[max(0, total - n):]:
        for param in layer.parameters():
            param.requires_grad = True

class MultimodalModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.text_model = AutoModel.from_pretrained(config.TEXT_MODEL_NAME)
        self.image_model = timm.create_model(config.IMAGE_MODEL_NAME, pretrained=True, num_classes=0)
        self.text_proj = nn.Sequential(
            nn.Linear(self.text_model.config.hidden_size, config.HIDDEN_DIM),
            nn.ReLU(),
            nn.LayerNorm(config.HIDDEN_DIM),
            nn.Dropout(config.DROPOUT)
        )
        self.image_proj = nn.Sequential(
            nn.Linear(self.image_model.num_features, config.HIDDEN_DIM),
            nn.ReLU(),
            nn.LayerNorm(config.HIDDEN_DIM),
            nn.Dropout(config.DROPOUT)
        )
        fused_dim = config.HIDDEN_DIM * 3 + 1
        self.regressor = nn.Sequential(
            nn.Linear(fused_dim, config.HIDDEN_DIM),
            nn.ReLU(),
            nn.LayerNorm(config.HIDDEN_DIM),
            nn.Dropout(config.DROPOUT),
            nn.Linear(config.HIDDEN_DIM, config.HIDDEN_DIM // 2),
            nn.ReLU(),
            nn.Linear(config.HIDDEN_DIM // 2, 1)
        )

        # === PARTIAL FREEZE TEXT =============================
        freeze_all(self.text_model)
        unfreeze_last_n_layers(self.text_model, n=6)

        # === PARTIAL FREEZE IMAGE ===========================
        freeze_all(self.image_model)
        if hasattr(self.image_model, "blocks"):
            for block in getattr(self.image_model, "blocks")[-5:]:
                for param in block.parameters():
                    param.requires_grad = True
        else:
            for param in self.image_model.parameters():
                param.requires_grad = True

    def forward(self, input_ids, attention_mask, image, mass=None,
                mask_text=False, mask_image=False):
        # TEXT
        if not mask_text:
            t = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
            t_emb = mean_pooling(t, attention_mask)
            t_emb = self.text_proj(t_emb)
        else:
            t_emb = torch.zeros((image.size(0), self.text_proj[0].out_features), device=image.device)

        # IMAGE
        if not mask_image:
            img_feat = self.image_model(image)
            i_emb = self.image_proj(img_feat)
        else:
            i_emb = torch.zeros((image.size(0), self.image_proj[0].out_features), device=image.device)

        interaction = t_emb * i_emb
        if mass is None:
            mass = torch.zeros((image.size(0), 1), device=image.device)
        else:
            mass = mass.view(-1, 1)

        fused = torch.cat([t_emb, i_emb, interaction, mass], dim=1)
        return self.regressor(fused).squeeze(1)

# src/utils.py
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from src.model import MultimodalModel
from src.dataset import collate_fn
from transformers import AutoTokenizer, get_cosine_schedule_with_warmup
import torch.cuda.amp as amp


def validate(model, loader, device, mask_text=False, mask_image=False, use_log_target=False):
    model.eval()
    mae_sum = 0.0
    count = 0
    mean = loader.dataset.mean
    std = loader.dataset.std
    with torch.no_grad():
        for batch in loader:
            preds = model(
                input_ids=batch["input_ids"].to(device),
                attention_mask=batch["attention_mask"].to(device),
                image=batch["image"].to(device),
                mass=batch["mass"].to(device),
                mask_text=mask_text,
                mask_image=mask_image
            )
            labels = batch["label"].to(device)
            # денормализация в той же шкале, в которой нормировали
            preds = preds * std + mean
            labels = labels * std + mean

            if use_log_target:
                # сейчас preds и labels в log1p-шкале, преобразуем обратно для MAE
                preds_orig = torch.expm1(preds).clamp(min=0.0)
                labels_orig = torch.expm1(labels).clamp(min=0.0)
                mae_sum += torch.sum(torch.abs(preds_orig - labels_orig)).item()
                count += labels_orig.size(0)
            else:
                mae_sum += torch.sum(torch.abs(preds - labels)).item()
                count += labels.size(0)
    return mae_sum / count

def train(config, train_ds, val_ds, use_log_target=False):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained(config.TEXT_MODEL_NAME)
    train_loader = DataLoader(
        train_ds,
        batch_size=config.BATCH_SIZE,
        shuffle=True,
        collate_fn=lambda b: collate_fn(b, tokenizer),
        num_workers=4,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_ds,
        batch_size=config.BATCH_SIZE,
        shuffle=False,
        collate_fn=lambda b: collate_fn(b, tokenizer),
        num_workers=2,
        pin_memory=True
    )
    model = MultimodalModel(config).to(device)

    # Фильтруем параметры по requires_grad
    # --- собрать параметры по группам ---
    text_params = []
    image_params = []
    head_params = []

    for name, p in model.named_parameters():
        if not p.requires_grad:
            continue
        # примерно по имени разделяем
        if name.startswith("text_model"):
            text_params.append(p)
        elif name.startswith("image_model"):
            image_params.append(p)
        else:
            head_params.append(p)

    optimizer_grouped_parameters = []
    if len(text_params) > 0:
        optimizer_grouped_parameters.append({"params": text_params, "lr": config.TEXT_LR})
    if len(image_params) > 0:
        optimizer_grouped_parameters.append({"params": image_params, "lr": config.IMAGE_LR})
    if len(head_params) > 0:
        optimizer_grouped_parameters.append({"params": head_params, "lr": config.HEAD_LR})

    optimizer = AdamW(optimizer_grouped_parameters, weight_decay=1e-3, eps=1e-8)
    total_steps = len(train_loader) * config.EPOCHS
    num_warmup_steps = int(0.2 * total_steps)  # 10% warmup
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=len(train_loader) * config.EPOCHS
    )

    criterion = nn.SmoothL1Loss()
    scaler = amp.GradScaler()  # mixed precision

    best_mae = 1e9
    for epoch in range(config.EPOCHS):
        model.train()
        total_loss = 0.0
        for batch in train_loader:
            optimizer.zero_grad()
            inputs = {
                "input_ids": batch["input_ids"].to(device),
                "attention_mask": batch["attention_mask"].to(device),
                "image": batch["image"].to(device),
                "mass": batch["mass"].to(device)
            }
            labels = batch["label"].to(device)
            with amp.autocast(enabled=(device=="cuda")):
                preds = model(**inputs)
                loss = criterion(preds, labels)
            scaler.scale(loss).backward()

            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
        scheduler.step()

        val_mae = validate(model, val_loader, device, mask_text=False, mask_image=False, use_log_target=use_log_target)
        mae_no_text = validate(model, val_loader, device, mask_text=True, mask_image=False, use_log_target=use_log_target)
        mae_no_image = validate(model, val_loader, device, mask_text=False, mask_image=True, use_log_target=use_log_target)
        print(f"Эпоха {epoch+1}/{config.EPOCHS}. Train Loss: {total_loss/len(train_loader):.4f}. Общее MAE: {val_mae:.2f}. MAE для изображений: {mae_no_text:.2f}. MAE для текста: {mae_no_image:.2f}")
        if val_mae < best_mae:
            best_mae = val_mae
            torch.save(model.state_dict(), config.SAVE_PATH)
            print(f"Сохранена модель MAE: {val_mae:.2f}")
    print(f"Было достигнуто лучшее MAE: {best_mae:.2f}")


/home/user/Projects/MultimodalModel/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Этап 3. Обучение модели

In [2]:
# main.py
import pandas as pd
import numpy as np
from src.dataset import MultimodalDataset
from src.utils import train
from src.config import Config
from sklearn.model_selection import train_test_split

def main():
    cfg = Config()

    dishes = pd.read_csv("data/dish.csv")
    ingredients = pd.read_csv("data/ingredients.csv")

    # Опционально: отсечь экстремальные выбросы
    dishes = dishes[dishes["total_calories"] < 750].reset_index(drop=True)
    dishes = dishes[dishes["total_mass"] < 900]

    train_df = dishes[dishes["split"] == "train"].reset_index(drop=True)
    # test_df = dishes[dishes["split"] == "test"].reset_index(drop=True)

    # Используем log1p трансформацию таргета — это часто стабилизирует обучение при сильной скошенности
    use_log = True

    if use_log:
        train_targets = np.log1p(train_df["total_calories"].clip(lower=0).values)
    else:
        train_targets = train_df["total_calories"].values

    train_mean = train_targets.mean()
    train_std = train_targets.std()

    # Масса как дополнительная числовая фича
    mass_vals = train_df["total_mass"].values
    mass_mean = mass_vals.mean()
    mass_std = mass_vals.std()
    
    train_df, val_df = train_test_split(
        train_df, 
        test_size=0.2,  
        random_state=42,  
        shuffle=True
    )

    train_ds = MultimodalDataset(train_df, ingredients, cfg, ds_type="train",
                                 target_mean=train_mean, target_std=train_std,
                                 mass_mean=mass_mean, mass_std=mass_std,
                                 use_log_target=use_log)
    
    val_ds = MultimodalDataset(val_df, ingredients, cfg, ds_type="test",
                               target_mean=train_mean, target_std=train_std,
                               mass_mean=mass_mean, mass_std=mass_std,
                               use_log_target=use_log)

    train(cfg, train_ds, val_ds, use_log_target=use_log)

if __name__ == "__main__":
    main()


/home/user/Projects/MultimodalModel/src/utils.py:100: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()  # mixed precision
/home/user/Projects/MultimodalModel/src/utils.py:115: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(enabled=(device=="cuda")):


Эпоха 1/30. Train Loss: 0.4627. Общее MAE: 161.61. MAE для изображений: 160.24. MAE для текста: 157.87
Сохранена модель MAE: 161.61
Эпоха 2/30. Train Loss: 0.4293. Общее MAE: 144.76. MAE для изображений: 154.26. MAE для текста: 137.88
Сохранена модель MAE: 144.76
Эпоха 3/30. Train Loss: 0.3635. Общее MAE: 124.29. MAE для изображений: 147.92. MAE для текста: 121.72
Сохранена модель MAE: 124.29
Эпоха 4/30. Train Loss: 0.3004. Общее MAE: 108.22. MAE для изображений: 141.21. MAE для текста: 120.08
Сохранена модель MAE: 108.22
Эпоха 5/30. Train Loss: 0.2539. Общее MAE: 97.67. MAE для изображений: 134.64. MAE для текста: 128.61
Сохранена модель MAE: 97.67
Эпоха 6/30. Train Loss: 0.2138. Общее MAE: 91.07. MAE для изображений: 128.91. MAE для текста: 118.70
Сохранена модель MAE: 91.07
Эпоха 7/30. Train Loss: 0.1855. Общее MAE: 87.38. MAE для изображений: 127.82. MAE для текста: 115.41
Сохранена модель MAE: 87.38
Эпоха 8/30. Train Loss: 0.1652. Общее MAE: 82.83. MAE для изображений: 125.57. MAE

# Этап 4. Валидация качества

In [5]:
import numpy as np
import torch

from src.dataset import MultimodalDataset
from src.model import MultimodalModel
from transformers import AutoTokenizer
from PIL import Image
import torchvision.transforms as T    
from src.config import Config
import pandas as pd
import random


def test_single_prediction(config, image_path, text, model_path, target_std, target_mean):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Загружаем модель
    model = MultimodalModel(config).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # Загружаем токенизатор
    tokenizer = AutoTokenizer.from_pretrained(config.TEXT_MODEL_NAME)

    # Подготовка изображения
    transform = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    # Подготовка текста
    text_encoded = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # Предсказание
    with torch.no_grad():
        pred = model(
            input_ids=text_encoded['input_ids'].to(device),
            attention_mask=text_encoded['attention_mask'].to(device),
            image=image_tensor
        )
    pred = pred * target_std + target_mean
    pred = torch.expm1(pred).clamp(min=0.0)

    print(f"Текст: {text}")
    print(f"Изображение: {image_path}")
    print(f"Предсказание: {pred.item():.2f}")
    return pred.item()



if __name__ == "__main__":
    cfg = Config()

    dishes = pd.read_csv("data/dish.csv")
    ingredients = pd.read_csv("data/ingredients.csv")

    test_df = dishes[dishes["split"] == "test"].reset_index(drop=True)

    test_ds = MultimodalDataset(test_df, ingredients, cfg, ds_type="test")
    train_targets = np.log1p(test_df["total_calories"].clip(lower=0).values)
    test_mean = train_targets.mean()
    test_std = train_targets.std()

    # Выбираем 5 случайных индексов
    random_indices = random.sample(range(len(test_ds)), min(5, len(test_ds)))

    print("Предсказания для 5 случайных блюд из тестового набора:")
    print("=" * 60)

    mae_errors = []
    for i, idx in enumerate(random_indices):
        # Получаем элемент из датасета
        item = test_ds[idx]

        # Получаем соответствующую строку из DataFrame для истинных значений
        row = test_df.iloc[idx]

        # Формируем путь к изображению
        image_path = f"data/images/{row['dish_id']}/rgb.png"
        
        # Получаем предсказание
        prediction = test_single_prediction(
            config=cfg,
            image_path=image_path,
            text=item["text"],
            model_path="best_model.pth", 
            target_std=test_std,
            target_mean=test_mean,
        )

        # Выводим сравнение с истинным значением
        true_calories = row['total_calories']
        error = abs(prediction - true_calories)
        mae_errors.append(error)
        print(f"Правильный ответ: {true_calories:.2f}")
        print(f"Ошибка предсказания: {error:.2f} калорий")
        print("=" * 60)

    final_mae = sum(mae_errors) / len(mae_errors)
    print(f"\nСредний MAE по {len(mae_errors)} примерам: {final_mae:.2f} калорий")

Предсказания для 5 случайных блюд из тестового набора:
Текст: List of ingredients: white rice
Total mass: 43.0 grams.
Изображение: data/images/dish_1558635523/rgb.png
Предсказание: 78.48
Правильный ответ: 55.90
Ошибка предсказания: 22.58 калорий
Текст: List of ingredients: eggplant, chicken thighs, mushroom, vinegar, cherry tomatoes, lime, mixed greens, spinach (cooked), garlic, shallots, lemon juice, onions, olive oil, pepper, quinoa, wine, carrot, cheese pizza, salt, lettuce, cucumbers, squash
Total mass: 188.0 grams.
Изображение: data/images/dish_1562099134/rgb.png
Предсказание: 276.96
Правильный ответ: 271.64
Ошибка предсказания: 5.32 калорий
Текст: List of ingredients: apple
Total mass: 116.0 grams.
Изображение: data/images/dish_1558639440/rgb.png
Предсказание: 69.36
Правильный ответ: 60.32
Ошибка предсказания: 9.04 калорий
Текст: List of ingredients: olive oil, blueberries, vinaigrette, walnuts, salt, cucumbers, onions, olives, feta cheese, arugula, vinegar, chard, pepper, cherry